# - Data Warehouse -

### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [1]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [2]:
awProductdf = run_query("SELECT * FROM Production.Product", "AdventureWorks2019")
awProductInventorydf = run_query("SELECT * FROM Production.ProductInventory", "AdventureWorks2019")
awProductSubcategorydf = run_query("SELECT * FROM Production.ProductSubcategory", "AdventureWorks2019")
awproductCategorydf = run_query("SELECT * FROM Production.ProductCategory", "AdventureWorks2019")

AenCProductdf = run_query("SELECT * FROM PRODUCT", "AenC")

nwProductsdf = run_query("SELECT * FROM products", "northwind")


Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

awProductSubcategorydf.columns

In [3]:
# Merge awProductSubCategoryIDdf and awProductCategorydf
merged_df1 = pd.merge(awProductSubcategorydf, awproductCategorydf, on='ProductCategoryID', how='outer')

merged_df1.reset_index(drop=True, inplace=True)
merged_df1.index = merged_df1.index + 1

# Rename the index to 'PrimaryKey'
merged_df1.index.name = 'IDSK'






In [4]:
awProductdf.reset_index(drop=True, inplace=True)
awProductdf.index = awProductdf.index + 1
awProductdf.index.name = 'IDSK'

In [5]:
merged_df2 = pd.merge(merged_df1, awProductdf, left_on='IDSK', right_on='ProductID', how='outer') # type: ignore

merged_df2['ProductID'] = merged_df2['ProductID'].astype('object')

# Reset the index and start from 1
merged_df2.reset_index(drop=True, inplace=True)
merged_df2.index = merged_df2.index + 1

# Rename the index to 'PrimaryKey'
merged_df2.index.name = 'IDSK'




In [6]:
AenCProductdf.reset_index(drop=True, inplace=True)
AenCProductdf.index = AenCProductdf.index + 1

# Rename the index to 'PrimaryKey'
AenCProductdf.index.name = 'IDSK'

In [7]:
merged_df2 = pd.merge(merged_df1, AenCProductdf, left_on='IDSK', right_on='IDSK', how='outer') # type: ignore



# Reset the index and start from 1
merged_df2.reset_index(drop=True, inplace=True)
merged_df2.index = merged_df2.index + 1

# Rename the index to 'PrimaryKey'
merged_df2.index.name = 'IDSK'




In [8]:
nwProductsdf.reset_index(drop=True, inplace=True)
nwProductsdf.index = nwProductsdf.index + 1

# Rename the index to 'PrimaryKey'
nwProductsdf.index.name = 'IDSK'

In [29]:
merged_df3 = pd.merge(merged_df2,nwProductsdf , left_on='IDSK', right_on='ProductID', how='inner') # type: ignore



# Reset the index and start from 1
merged_df3.reset_index(drop=True, inplace=True)
merged_df3.index = merged_df3.index + 1

# Rename the index to 'PrimaryKey'
merged_df3.index.name = 'IDSK'


merged_df3

,ProductSubcategoryID,ProductCategoryID,Name_x,rowguid_x,ModifiedDate_x,Name_y,rowguid_y,ModifiedDate_y,id,name,...,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
IDSK,,,,,,,,,,,,,,,,,,,,,
1,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30,300,Tee Shirt,...,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,False
2,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30,301,Tee Shirt,...,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,False
3,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30,302,Tee Shirt,...,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,False
4,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30,400,Baseball Cap,...,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,False
5,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30,401,Baseball Cap,...,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,True
6,6,2,Brakes,D43BA4A3-EF0D-426B-90EB-4BE4547DD30C,2008-04-30,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30,500,Visor,...,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.00,120,0,25,False
7,7,2,Chains,E93A7231-F16C-4B0F-8C41-C73FDEC62DA0,2008-04-30,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30,501,Visor,...,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00,15,0,10,False
8,8,2,Cranksets,4F644521-422B-4F19-974A-E3DF6102567E,2008-04-30,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30,600,Sweatshirt,...,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.00,6,0,0,False
9,9,2,Derailleurs,1830D70C-AA2A-40C0-A271-5BA86F38F8BF,2008-04-30,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30,601,Sweatshirt,...,9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.00,29,0,0,True


In [9]:


merged_df3.columns

Index(['ProductSubcategoryID', 'ProductCategoryID', 'Name_x', 'rowguid_x',
       'ModifiedDate_x', 'Name_y', 'rowguid_y', 'ModifiedDate_y', 'id', 'name',
       'description', 'prod_size', 'color', 'quantity', 'unit_price',
       'picture_name', 'Category', 'ProductID', 'ProductName', 'SupplierID',
       'CategoryID', 'QuantityPerUnit', 'UnitPrice', 'UnitsInStock',
       'UnitsOnOrder', 'ReorderLevel', 'Discontinued'],
      dtype='object')

#### 3 - Data loading

In [41]:

Productquery = """
CREATE TABLE Product (
    ProductSubcategoryID INT,
    ProductCategoryID INT,
    id INT,
    Quantity INT,
    Category VARCHAR(255),
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(255),
    SupplierID INT,
    CategoryID INT,
    UnitsInStock INT,
    Discontinued VARCHAR(255),
);
"""

# Create the table in SQL Server
db_name = "MustafaTest"
conn, cursor = create_connection(db_name)
cursor.execute(Productquery)
conn.commit()
conn.close()


In [42]:
merged_df3.columns

Index(['ProductSubcategoryID', 'ProductCategoryID', 'Name_x', 'rowguid_x',
       'ModifiedDate_x', 'Name_y', 'rowguid_y', 'ModifiedDate_y', 'id', 'name',
       'description', 'prod_size', 'color', 'quantity', 'unit_price',
       'picture_name', 'Category', 'ProductID', 'ProductName', 'SupplierID',
       'CategoryID', 'QuantityPerUnit', 'UnitPrice', 'UnitsInStock',
       'UnitsOnOrder', 'ReorderLevel', 'Discontinued'],
      dtype='object')

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [47]:
def clean_row(row):
    try:
        row['ProductSubcategoryID'] = int(row['ProductSubcategoryID'])
    except (ValueError, TypeError):
        row['ProductSubcategoryID'] = 0

    try:
        row['ProductCategoryID'] = int(row['ProductCategoryID'])
    except (ValueError, TypeError):
        row['ProductCategoryID'] = 0

    try:
        row['id'] = int(row['id'])
    except (ValueError, TypeError):
        row['id'] = 0

    try:
        row['ProductID'] = int(row['ProductID'])
    except (ValueError, TypeError):
        row['ProductID'] = 0

    try:
        row['SupplierID'] = int(row['SupplierID'])
    except (ValueError, TypeError):
        row['SupplierID'] = 0

    try:
        row['CategoryID'] = int(row['CategoryID'])
    except (ValueError, TypeError):
        row['CategoryID'] = 0

    try:
        row['UnitsInStock'] = float(row['UnitsInStock'])
    except (ValueError, TypeError):
        row['UnitsInStock'] = 0.0

    try:
        row['Discontinued'] = 1 if row['Discontinued'] else 0
    except (ValueError, TypeError):
        row['Discontinued'] = 0

    try:
        row['quantity'] = int(row['quantity'])
    except (ValueError, TypeError):
        row['quantity'] = 0

    row['Category'] = str(row['Category'])
    row['ProductName'] = str(row['ProductName'])

    return row

# Establish the connection
test_database_name = 'MustafaTest'
conn, cursor = create_connection(test_database_name)

# Iterate through the filtered DataFrame, clean each row, and insert into the database
for index, row in merged_df3.iterrows():
    try:
        row = clean_row(row)

        # SQL insert query using parameterized query
        query = """
            INSERT INTO Product (
                ProductSubcategoryID, ProductCategoryID, id, quantity, Category, 
                ProductID, ProductName, SupplierID, CategoryID, UnitsInStock, Discontinued
            ) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        # Data to insert
        values = (
            row['ProductSubcategoryID'], row['ProductCategoryID'], row['id'], row['quantity'], row['Category'], 
            row['ProductID'], row['ProductName'], row['SupplierID'], row['CategoryID'], row['UnitsInStock'], row['Discontinued']
        )
        
        # Execute the query
        cursor.execute(query, values)
    except Exception as e:
        print(f"Error at index {index}: {e}")

# Commit the changes and close the connection
conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [26]:
# Controleren op ontbrekende waarden
missing_values = merged_df3.isnull().sum()

# Controleren op duplicaten
duplicate_rows = merged_df3.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)

Aantal ontbrekende waarden: ProductSubcategoryID     0
ProductCategoryID        0
Name_x                   0
rowguid_x                0
ModifiedDate_x           0
Name_y                   0
rowguid_y                0
ModifiedDate_y           0
id                      27
name                    27
description             27
prod_size               27
color                   27
quantity                27
unit_price              27
picture_name            27
Category                27
ProductID                0
ProductName              0
SupplierID               0
CategoryID               0
QuantityPerUnit          0
UnitPrice                0
UnitsInStock             0
UnitsOnOrder             0
ReorderLevel             0
Discontinued             0
dtype: int64
Aantal duplicaten: 0


**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.